In [145]:
# hide
# default_exp params

# Notebook Parameter Management

Having the ability to paramterise notebook executions is very useful..

In [147]:
# export

from pathlib import Path

import nbformat
from nbdev.export import (Config, find_default_export, nbglob, params_cell,
                          read_nb)
from nbformat.notebooknode import NotebookNode

# Notebook utilities 

> Used to create & access the parameters cell.

In [148]:
# export


def find_params_cell(nb: NotebookNode):
    params_cell = [c for c in nb["cells"] if c["metadata"] == {"tags": ["parameters"]}]
    return params_cell

In [149]:
import os

sample_nb = os.path.join(Path(".").resolve().parent, "examples", "top2vec.ipynb")

In [152]:
assert len(find_params_cell(read_nb(Path(sample_nb)))) == 1
assert len(find_params_cell(read_nb(Path("index.ipynb")))) == 0

In [153]:
# export

DEFAULT_PARAMS_CELL = {
    "cell_type": "code",
    "execution_count": None,
    "metadata": {"tags": ["parameters"]},
    "outputs": [],
    "source": "# parameters\n",
}

In [154]:
# export


def add_missing_params_cell(nb_path: Path, persist: bool = True):
    nb = read_nb(nb_path)
    if len(find_params_cell(nb)) > 0:
        print(f"Skipping {nb_path} already has parameters cell")
        return
    nb["cells"].insert(0, nbformat.from_dict(DEFAULT_PARAMS_CELL))
    if persist:
        nbformat.write(nb, nb_path)
    return nb

In [155]:
with_params = os.path.join(Path(".").resolve().parent, "examples", "top2vec.ipynb")
without_params = os.path.join(
    Path(".").resolve().parent, "examples", "top2vec-no-params.ipynb"
)

add_missing_params_cell(with_params, False)
assert len(find_params_cell(read_nb(without_params))) == 0
parameterised_nb = add_missing_params_cell(without_params, False)
assert len(find_params_cell(parameterised_nb)) == 1

Skipping /home/jovyan/git/scidev/examples/top2vec.ipynb already has parameters cell


In [156]:
# export


def extract_params_to_file(nb_path: Path, params_file_path: Path):
    with open(params_file_path, "w") as params_file:
        params_file.writelines(params_cell[0]["source"])

In [157]:
# export


def list_mod_files(files):
    modules = []
    for f in files:
        fname = Path(f)
        nb = read_nb(fname)
        default = find_default_export(nb["cells"])
        if default is not None:
            default = os.path.sep.join(default.split("."))
            modules.append(default)
    return modules

In [158]:
# export


def extract_as_files(suffix="_params.py"):
    nbs = nbglob(recursive=True)
    param_files = list_mod_files(nbs)
    params_files = [
        Path(os.path.join(Config().path("lib_path"), pf + suffix)) for pf in param_files
    ]
    for nb_path, pf_path in zip(nbs, params_files):
        extract_params_to_file(nb_path, pf_path)

In [159]:
# export


def extract_params_as_dict(params_file_path: Path):
    params = {}
    with open(params_file_path, "r") as params_file:
        for line in params_file:
            if line.startswith("#"):
                continue
            (key, val) = line.split("=")
            params[key.strip()] = val.strip('\n "')
    return params